In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
import pickle

In [24]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 2000)

In [4]:
df_country_code = pd.read_csv('csv_files/wikipedia-iso-country-codes.csv')
df_world_cities = pd.read_csv('csv_files/worldcitiespop.csv', dtype='str') 

In [5]:
df_country_code.shape

(246, 5)

In [6]:
df_country_code.head(3)

,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL


In [7]:
df_country_code.columns = [x.lower() for x in df_country_code.columns]

In [8]:
df_world_cities.shape

(3173958, 7)

In [9]:
df_world_cities.head(3)

,Country,City,AccentCity,Region,Population,Latitude,Longitude
0,ad,aixas,Aixàs,06,NaN,42.4833333,1.4666667
1,ad,aixirivali,Aixirivali,06,NaN,42.466666700000005,1.5
2,ad,aixirivall,Aixirivall,06,NaN,42.466666700000005,1.5


In [10]:
df_world_cities.columns = [x.lower() for x in df_world_cities.columns]

### alpha-2_code to lower() and merge country code_2 with df_world_cities 

In [11]:
df_country_code['alpha-2 code'] = df_country_code['alpha-2 code'].str.lower()

In [12]:
df_world_cities = df_world_cities.merge(df_country_code[['alpha-2 code', 'english short name lower case']], how='left', left_on='country', right_on='alpha-2 code')

In [34]:
df_world_cities

,country_code,city,latitude,longitude,country
0,ad,Aixas,42.4833333,1.4666667,Andorra
1,ad,Aixirivali,42.466666700000005,1.5,Andorra
2,ad,Aixirivall,42.466666700000005,1.5,Andorra
3,ad,Aixirvall,42.466666700000005,1.5,Andorra
4,ad,Aixovall,42.466666700000005,1.4833333,Andorra
...,...,...,...,...,...
3173953,zw,Zimre Park,-17.8661111,31.213611100000005,Zimbabwe
3173954,zw,Ziyakamanas,-18.216666699999998,27.95,Zimbabwe
3173955,zw,Zizalisari,-17.7588889,31.010555600000004,Zimbabwe
3173956,zw,Zuzumba,-20.033333300000002,27.933333299999997,Zimbabwe


In [14]:
df_world_cities.drop(columns=['accentcity', 'population', 'region', 'alpha-2 code'], inplace=True)

In [15]:
df_world_cities.columns = ['country_code', 'city', 'latitude', 'longitude', 'country']

In [16]:
df_world_cities.city = df_world_cities.city.str.title()

### Find and replace no utf symbols

In [17]:
def find_no_utf_symbols(sr: pd.Series):
    find_symols = r'[^.a-zA-Z0-9)(&"\s>;<=,-/}{\']'
    rare_symbols = sr.map(lambda x: '-'.join(re.findall(find_symols, str(x))))
    rare_symbols = ['-'.join(set(x.split('-'))) for x in list(set(rare_symbols))]
    rare_symbols = [x.split('-') for x in rare_symbols]
    
    all_symbs = []
    for symbols in rare_symbols:
        for symb in symbols:
            if symb != '':
                all_symbs.append(symb)
    list_symbols = list(set(all_symbs))
    
    return list_symbols

In [18]:
rare_symbols = find_no_utf_symbols(df_world_cities.city)

In [23]:
# rare_symbols

In [19]:
# replace some rare_symbols
rare_symbols_keys = {'Þ':'p', 'ü':'u', 'ñ':'n', 'Ð':'D', 'ð':'d'}

def replace_rare_symbols(df: pd.DataFrame, col_name: str, dict_obj: dict):
    for k, v in rare_symbols_keys.items():
        df[col_name] = df[col_name].map(lambda x: x.replace(k, v) if re.search(k, str(x)) else x)
    return df[col_name] 

In [24]:
df_world_cities.city = replace_rare_symbols(df_world_cities, 'city', rare_symbols_keys)

In [25]:
rare_symbols = find_no_utf_symbols(df_world_cities.city)

In [26]:
def remove_rare_symbols(sr: pd.Series, rare_symbols: list):
    for symb in rare_symbols:
        sr = sr.map(lambda x: x.replace(symb, '') if symb in str(x) else x)
    return sr       

In [27]:
df_world_cities.city = remove_rare_symbols(df_world_cities.city, rare_symbols)

In [29]:
rare_symbols_country = find_no_utf_symbols(df_world_cities['country'])

In [30]:
rare_symbols_country

['ô']

In [33]:
rare_symbols_keys = {'ô':'o'}
df_world_cities.country = replace_rare_symbols(df_world_cities, 'country', rare_symbols_keys)

In [41]:
df_world_cities.dropna(inplace=True)

In [1]:
from Lat_lon.lat_lon import *

In [2]:
palmyra = LatLon(Latitude(8.88), Longitude(-162.03)) # Location of Palmyra Atoll in decimal degrees
# palmyra = LatLon(5.8833, -162.0833) # Same thing but simpler!
# palmyra = LatLon(Latitude(degree = 5, minute = 52, second = 59.88), Longitude(degree = -162, minute = -4.998) # or more complicated!
tmp = palmyra.to_string('d% %m% %S% %H') # Print coordinates to degree minute second
# ('5 52 59.88 N', '162 4 59.88 W')

In [3]:
tmp

('8 52 48 N', '162 1 48 W')

In [63]:
# with open('pickle_files/df_teams_data', 'wb') as f:
#     pickle.dump(df_for_add, f)

In [2]:
palmyra = LatLon(Latitude(5.8833), Longitude(-162.0833))

In [3]:
palmyra

Latitude 5.8833, Longitude -162.0833

In [60]:
a=60
b=90
print(cmp(a,b))

NameError: name 'cmp' is not defined